#### Load production model from UC and perform inference

In [0]:
%pip install mlflow==3.0.1 xgboost==3.0.0
%restart_python

In [0]:
import mlflow
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as func
from pyspark.sql.functions import col

In [0]:
dbutils.widgets.text('catalog_name','','Enter catalog name')
dbutils.widgets.text('schema_name','','Enter schema name')
dbutils.widgets.text('model_name','','Enter model name')
dbutils.widgets.text('suffix','','Enter table suffix')

In [0]:
#dbutils.widgets.removeAll()

In [0]:
catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
suffix = dbutils.widgets.get('suffix')
table_name = f"advanced_churn_bronze_customers_{suffix}"
uc_model_name = f"{catalog_name}.{schema_name}.{dbutils.widgets.get('model_name')}_{suffix}"
uc_location = f"{catalog_name}.{schema_name}.{table_name}"
print(f"UC location: {uc_location}\nModel name: {uc_model_name}")

In [0]:
spark_df = spark.table(uc_location)
display(spark_df)

In [0]:
loaded_model = mlflow.pyfunc.spark_udf(spark, f"models:/{uc_model_name}@production", env_manager="local",result_type=DoubleType())

predictions_spark = spark_df.withColumn('predictions', loaded_model(func.struct(*map(col, spark_df.columns))))
display(predictions_spark)